In [3]:
from scipy.spatial import  distance as dist #to find dist of co-ordinates
from imutils import face_utils #get the co-ordinates of different parts
import imutils #resize
import dlib #face detection & landmark on face
import cv2 #frame acquit
import winsound
frequency = 2500
duration = 1000

In [ ]:
def eyeAspectRatio (eye):
  #vertcal
  A= dist.euclidean(eye[1],eye[5])
  B= dist.euclidean(eye[2],eye[4])
  #horizontal
  C= dist.euclidean(eye[0],eye[3])
  ear = (A+B) / (2.0*C)
  return ear

In [ ]:
count=0
earThresh= 0.3 #distance between vertical eye co-ordinates threshold
earFrames= 48 # consecutive frames for eye closure
shapePredictor = "shape_predictor_68_landmark.dat"

In [ ]:
cam= cv2.VideoCapture(1)
detector= dlib.get_frontal_face_detector()
predictor= dlib.shape_predictor(shapePredictor)

In [ ]:
#get the co-ordinates of left and right eye
(lStart,lEnd)= face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart,rEnd)= face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
while True:
  _,frame = cam.read() #read frames from camera
  frames= imutils.resize(frame,width=450)  #resizing
  gray=cv2.cvtcolor(frame,cv2.COLOUR_BGR2GRAY)  #colour 2 gray
  rects = detector(gray,0)  #detect the face
   
  for rect in rects:
    shape= predictor(gray,rect)
    shape= face_utils.shape_to_np(shape)
    
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eyeAspectRatio(leftEye)
    rightEAR = eyeAspectRatio(rightEye)

    ear= (leftEAR+rightEAR)/2.0

    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    cv2.drawContous(frame,[leftEyeHull],-1,(0,0,255),1)
    cv2.drawContours(frame,[rightEyeHull],-1,(0,0,255),1)


    if ear<earThresh:
      count+=1
      if count>= earFrames:
        cv2.putText(frame,"DROWSINESS_DETECTED",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        winsound.Beep(frequency,duration)
      else:
        count = 0
  cv2.imshow("Frame",frame)
  key= cv2.waitKey(1) & 0xFF
  if key == ord("q"):
    break


In [ ]:
cam.release()
cv2.destroyAllWindows()